In [1]:
# Import modules
import cv2
import numpy as np
import os
#from google.colab import drive

# Mount google drive
#drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
# Set categories in dataset images
categories = ["Faces", "cougar_face", "flamingo", "nautilus", "sunflower", "Faces_easy", "dalmatian", "headphone", "pagoda", "tick", "brontosaurus", "dollar_bill", "lotus", "soccer_ball", "windsor_chair", "car_side", "dragonfly", "menorah", "stop_sign", "yin_yang"]

# Configure paths
#base_path = "/content/gdrive/My Drive/Colab Notebooks"
base_path = os.getcwd()
data_path = base_path + "/dataset"

In [3]:
# Initialize detector
detector = cv2.ORB_create()

In [4]:
# Set empty arrays
train_path = []
train_label = []
train_features = np.array([])
tmp_stack = []

# Set # of total images in one category
image_length = 30
count = 0

In [5]:
# Extract features in dataset images by opencv detector
for index, item in enumerate(categories):
  
  category_dir = data_path + "/train/" + item

  for index2 in range(image_length):

    image_path = category_dir + "/image_%04d.jpg" % (index2+1)
    train_path.append(image_path)
    train_label.append(index)
    img = cv2.imread(image_path)

    gimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    kpt, desc = detector.detectAndCompute(gimg, None)

    if train_features.size == 0:
      train_features = np.float32(desc)
    else:
      train_features = np.append(train_features, np.float32(desc), axis=0)

    count += 1
    print('%d/%d - %s - %d feature points are detected\n' % (count, image_length*20, image_path, desc.shape[0]))

1/600 - /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0001.jpg - 492 feature points are detected

2/600 - /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0002.jpg - 500 feature points are detected

3/600 - /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0003.jpg - 495 feature points are detected

4/600 - /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0004.jpg - 500 feature points are detected

5/600 - /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0005.jpg - 500 feature points are detected

6/600 - /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0006.jpg - 500 feature points are detected

7/600 - /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0007.jpg - 500 feature points are detected

8/600 - /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0008.jpg - 498 feature points are detected

9/600 - /content/gdrive/My Drive/Colab Notebooks

In [6]:
# Create dictionary based on extracted features
dictionary_size = 10
dict_path = base_path + "/5dictionary.npy"

# In this part, K-means clustering can be used
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.1)
ret, label, dictionary = cv2.kmeans(train_features, dictionary_size, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

# Save dictionary as file
np.save(dict_path, dictionary)

In [7]:
# Generate K-Nearest model based on dictionary
knn = cv2.ml.KNearest_create()
knn.train(dictionary, cv2.ml.ROW_SAMPLE, np.float32(range(dictionary_size)))
train_desc = np.float32(np.zeros((len(train_path), dictionary_size)))

In [8]:
# Train BOW
for index, path in enumerate(train_path):

  img = cv2.imread(path)
  gimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  # Train BOW level 0
  kpt, desc = detector.detectAndCompute(gimg, None)
  ret, result, nbs, dist = knn.findNearest(np.float32(desc), k=1)

  hist, bins = np.histogram(np.int32(result), bins=range(dictionary_size + 1))
  train_desc[index, :] = 0.25 * np.float32(hist) / np.float32(np.sum(hist))

  width, height, channle = img.shape

  # Set width and height of sub-images in level 1
  lv1_ws = [0, int(width/2)]
  lv1_we = [int(width/2), width]
  lv1_hs = [0, int(height/2)]
  lv1_he = [int(height/2), height]

  temp = np.array([])

  # Train sub-images in level 1
  for ws, we in zip(lv1_ws, lv1_we):
    for hs, he in zip(lv1_hs, lv1_he):
      # Take parts from level 1 width-height lists.
      subimg = gimg[ws:we, hs:he]
      kpt, desc = detector.detectAndCompute(subimg, None)

      # If there is no detected feature, fill zeros.
      if desc is None:
        append_target = np.float32(np.zeros((dictionary_size, )))
      else:
        # If there is detected feature, get BOW histogram and normalize it
        ret, result, nbs, dist = knn.findNearest(np.float32(desc), k=1)
        hist, bins = np.histogram(np.int32(result), bins=range(dictionary_size + 1))
        append_target = 0.75 * np.float32(hist) / np.float32(np.sum(hist))
      
      temp = np.append(temp, append_target)

  tmp_stack.append(temp.tolist())
  print('%d/%d - Representing %s \n' % (index+1, len(train_path), path))

# concatenate histogram data
tmp_stack = np.array(np.float32(tmp_stack))
train_desc = np.hstack((train_desc, tmp_stack))

# Normalize concatenated histogram data
#for index in range(len(train_path)):
  #train_desc[index] = np.float32(train_desc[index]) / np.float32(np.sum(train_desc[index]))

1/600 - Representing /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0001.jpg 

2/600 - Representing /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0002.jpg 

3/600 - Representing /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0003.jpg 

4/600 - Representing /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0004.jpg 

5/600 - Representing /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0005.jpg 

6/600 - Representing /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0006.jpg 

7/600 - Representing /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0007.jpg 

8/600 - Representing /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0008.jpg 

9/600 - Representing /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0009.jpg 

10/600 - Representing /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0010.jpg 

11/600 - 

In [9]:
# Train SVM both RBF and INTERSECTION kernel
svm_model_path_RBF = base_path + "/5svmmodel_RBF.xml"
svm_model_path_INTER = base_path + "/5svmmodel_INTER.xml"

svm_RBF = cv2.ml.SVM_create()
svm_RBF.trainAuto(train_desc, cv2.ml.ROW_SAMPLE, np.array(train_label))
svm_RBF.save(svm_model_path_RBF)

svm_INTER = cv2.ml.SVM_create()
svm_INTER.setKernel(cv2.ml.SVM_INTER)
svm_INTER.trainAuto(train_desc, cv2.ml.ROW_SAMPLE, np.array(train_label))
svm_INTER.save(svm_model_path_INTER)

In [10]:
# Configure test data path and calculate # of total test data
test_path = data_path + "/test"
test_length = 5
total_test = test_length * len(categories)

# To calculate correctness
correct_RBF = 0
correct_INTER = 0

In [11]:
# Test BOW model
for index, item in enumerate(categories):

  target_dir = test_path + "/" + item

  for index2 in range(test_length):

    test_desc = np.float32(np.zeros((1, dictionary_size)))
    image_path = target_dir + "/image_%04d.jpg" % (31 + index2)

    # Get level 0 histogram of image
    img = cv2.imread(image_path)
    gimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kpt, desc = detector.detectAndCompute(gimg, None)
    ret, result, nbs, dist = knn.findNearest(np.float32(desc), k=1)
    hist, bins = np.histogram(np.int32(result), bins=range(dictionary_size + 1))
    hist = 0.25 * np.float32(hist) / np.float32(np.sum( hist))

    width, height, channle = img.shape

    # Divide image to 4 sub images
    lv1_ws = [0, int(width/2)]
    lv1_we = [int(width/2), width]
    lv1_hs = [0, int(height/2)]
    lv1_he = [int(height/2), height]

    temp = np.array([])

    # Get level 1 histogram of each sub images
    for ws, we in zip(lv1_ws, lv1_we):
      for hs, he in zip(lv1_hs, lv1_he):
        subimg = gimg[ws:we, hs:he]
        kpt, desc = detector.detectAndCompute(subimg, None)

        # If test sub-image features cannot be detected, fill zeros
        if desc is None:
          append_target = np.float32(np.zeros((dictionary_size, )))
        else:
          # Get level 1 sub-image histogram
          ret, result, nbs, dist = knn.findNearest(np.float32(desc), k=1)
          hist, bins = np.histogram(np.int32(result), bins=range(dictionary_size + 1))
          append_target = 0.75 * np.float32(hist) / np.float32(np.sum(hist))

        temp = np.append(temp, append_target)

    # Normalize level 0 and level 1 before concatenation
    temp = np.array([np.float32(temp)])
    test_desc = np.hstack((test_desc, temp))

    #test_desc = np.float32(test_desc) / np.float32(np.sum(test_desc))

    # Apply both RBF and intersection SVM kernel
    ret, result_RBF = svm_RBF.predict(test_desc)

    if item == categories[int(result_RBF[0][0])]:
      correct_RBF += 1

    ret, result_INTER = svm_INTER.predict(test_desc)

    if item == categories[int(result_INTER[0][0])]:
      correct_INTER += 1

In [12]:
# Get accuracy of RBF kernel
accuracy = correct_RBF / total_test
print(accuracy)

0.37


In [13]:
# Get accuracy of intersection kernel
accuracy = correct_INTER / total_test
print(accuracy)

0.35
